In [1]:
import requests
import pandas as pd
import os
from bs4 import BeautifulSoup

In [2]:
####################################################################


def get_usernames(doc,details_dict):
    us = doc.find_all('h1',{'class':'h3 lh-condensed'})
    for tag in us:
        details_dict['Username'].append(tag.text.strip())
    return

def get_personal_info(doc,details_dict):
    pi = doc.find_all('div',{'class':'col-md-7'})
    for tag in pi:
        details_dict['Info'].append(tag.text.strip())
    return   

#######################################################################
def parse_count(count):
    if(count[-1]=='k'):
        return int(float(count[:-1])*1000)
    else:
        return int(count)



def get_full_name(doc):
    fn = doc.find_all('span',{'class':'p-name vcard-fullname d-block overflow-hidden'})
    if(fn == []):
        fn_org = doc.find_all('h1',{'class':'h2 lh-condensed'})
        status = 1
        return fn_org[0].text.strip(),status
    status = 0 
    return fn[0].text.strip(),status


def get_followers(doc,status):
    if status==1:
        return '-'
    fl1 = doc.find_all('div',{'class':'mb-3'})
    fl2 = fl1[3].find_all('a',{'class':'Link--secondary no-underline no-wrap'})
    fl3 = fl2[0].find('span',{'class':'text-bold color-fg-default'}).text
    return(parse_count(fl3))

def get_following(doc,status):
    if status==1:
        return '-'
    fw1 = doc.find_all('div',{'class':'mb-3'})
    fw2 = fw1[3].find_all('a',{'class':'Link--secondary no-underline no-wrap'})
    fw3 = fw2[1].find('span',{'class':'text-bold color-fg-default'}).text
    return(parse_count(fw3))

def get_no_of_stars(doc,status):
    if status==1:
        return '-'
    fs1 = doc.find_all('div',{'class':'mb-3'})
    fs2 = fs1[3].find_all('a',{'class':'Link--secondary no-underline no-wrap'})
    fs3 = fs2[2].find('span',{'class':'text-bold color-fg-default'}).text
    return(parse_count(fs3))
    
def get_location(doc,status):
    if status==1:
        l = doc.find_all('span',{'itemprop':'location'})
        if(l==[]):
            return '-'
        return(l[0].text)
    loc = doc.find_all('span',{'class':'p-label'})
    if(loc==[]):
        return '-'
    return(loc[0].text.strip())

def get_no_of_repos(doc,status): 
    if status==1:
        return '-'
    repono = doc.find_all('span',{'class':'Counter'})
    return(parse_count(repono[0].text))

def get_no_of_con(doc,status):
    if status==1:
        return '-'
    conno = doc.find_all('h2',{'class':'f4 text-normal mb-2'})
    return(conno[0].text.split()[0])

###########################################################################

    

In [3]:
def detail_scrapper(url,details_dict):#beta
    response = requests.get(url)
    doc = BeautifulSoup(response.text, 'html.parser')
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(url))
    
    
    det = (get_full_name(doc))
    if(det[1] == 1):
        status = 1
    else:
        status = 0
      
    
    details_dict['Full Name'].append(det[0])    
    details_dict['Followers'].append(get_followers(doc,status))
    details_dict['Following'].append(get_following(doc,status))
    details_dict['No. of stars'].append(get_no_of_stars(doc,status))
    details_dict['Location'].append(get_location(doc,status))
    details_dict['Repositories'].append(get_no_of_repos(doc,status))
    details_dict['Contributions (Last Year)'].append(get_no_of_con(doc,status))
    return
    

def diver(doc,details_dict):#baap

    dive = doc.find_all('h1',{'class':'h3 lh-condensed'})
    for i in range(len(dive)):
        dive_url = 'https://github.com' + dive[i].find('a')['href']
        print("Collecting data about {}...".format(dive_url.split('/')[-1]))
        detail_scrapper(dive_url,details_dict)

    df = pd.DataFrame(details_dict)
    path = 'Github_Sponsors/{}.csv'.format('Github Sponsors Data')
    df.to_csv(path,index = None)
    print('Process Completed Successfully.')
    return 
   
def front():#sabkabaap
    sponsors_url = 'https://github.com/sponsors/community'
    response = requests.get(sponsors_url)
    doc = BeautifulSoup(response.text, 'html.parser')
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(sponsors_url))

    sponsor_details_dict = {
    'Username':[],'Info':[],'Full Name':[],'Followers':[],'Following': [],'No. of stars': [],'Location':[],'Repositories':[],'Contributions (Last Year)': []
    }
    os.makedirs('Github_Sponsors',exist_ok = True)
    get_usernames(doc,sponsor_details_dict)
    get_personal_info(doc,sponsor_details_dict)
    diver(doc,sponsor_details_dict)
    return doc
       
        
#################################
def individual_repos(url):
    response = requests.get(url)
    doc = BeautifulSoup(response.text, 'html.parser')
    if response.status_code != 200:
        raise Exception('Failed to load page {}'.format(url))
    
    user_repo_dict = {'Repo Name':[],'URL':[]}    
    #status 1
    repname = doc.find_all('h3',{'class':'wb-break-all'})
    for i in range(len(repname)):
        user_repo_dict['URL'].append('https://github.com' + repname[i].find('a')['href'])
        user_repo_dict['Repo Name'].append(repname[i].find('a').text.strip())
    
    if(url.split('/')[-1]!='repositories'):
        pname = url.split('/')[-1].split('?')[0]
    else:
        pname = url.split('/')[-2]
    path = 'Github_Sponsors/{}.csv'.format(pname)
    if os.path.exists(path):
        print("The file {} already exists. Skipping....".format(path))
        return
              

    pd.DataFrame(user_repo_dict).to_csv(path,index=None)
    return
            
            
    
        
        
def individual(doc):
    dive = doc.find_all('h1',{'class':'h3 lh-condensed'})
    for i in range(len(dive)):
        dive_url = 'https://github.com' + dive[i].find('a')['href']    
        response = requests.get(dive_url)
        doc = BeautifulSoup(response.text, 'html.parser')
        if response.status_code != 200:
            raise Exception('Failed to load page {}'.format(dive_url))
        
        print("Extracting data about {}'s repositories".format(dive_url.split('/')[-1]))
        det = (get_full_name(doc))
        if(det[1] == 1):
            status = 1
        else:
            status = 0
        
        if status == 0:
            repo_link = doc.find_all('div',{'class':'UnderlineNav width-full box-shadow-none'})
            rep = repo_link[0].find_all('nav',{'class':'UnderlineNav-body'})[0].find_all('a',{'class':'UnderlineNav-item'})[1]['href']
         
        else: 
            repo_link = doc.find_all('div',{'class':'width-full d-flex position-relative container-xl'})
            rep = repo_link[0].find_all('li',{'class':'d-flex js-responsive-underlinenav-item'})[1].find_all('a',{'class':'UnderlineNav-item'})[0]['href']
       

        url = 'https://github.com' + rep
        individual_repos(url)
    print('Process Completed Successfully.')
    return
 
def extract():
    individual(front())

            
        
        
        
        
    

In [5]:
extract()

Process Completed Successfully.
Extracting data about patricia-gallardo's repositories
Extracting data about claui's repositories
Extracting data about curl's repositories
Extracting data about eslint's repositories
Extracting data about EbookFoundation's repositories
Extracting data about potatoqualitee's repositories
Extracting data about M0nica's repositories
Extracting data about chaynHQ's repositories
Extracting data about CommandPost's repositories
Extracting data about aurelia's repositories
Extracting data about aeneasr's repositories
Extracting data about phalcon's repositories
Extracting data about nzakas's repositories
Extracting data about flarum's repositories
Extracting data about sanderstad's repositories
Extracting data about tenancy's repositories
Extracting data about katef's repositories
Extracting data about joshuaulrich's repositories
Extracting data about shentao's repositories
Extracting data about wifelette's repositories
Extracting data about jina's repositorie